<a href="https://colab.research.google.com/github/RebecaGis/Surface-Heat-Analysis/blob/main/surface_heat_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(shiny)
library(shinydashboard)
library(terra)
library(sf)
library(leaflet)
library(ggplot2)
library(shinyFiles)
library(fs)
library(osmdata)
library(dplyr)
library(purrr)
library(stringr)

# Aumentar o limite de upload para 500MB
options(shiny.maxRequestSize = 600 * 1024^2)

ui <- dashboardPage(
  dashboardHeader(title = "Surface Heat Analysis"),
  dashboardSidebar(
    sidebarMenu(
      menuItem("Upload de Arquivos", tabName = "upload", icon = icon("upload")),
      menuItem("Resultados", tabName = "results", icon = icon("map")),
      menuItem("Download", tabName = "download", icon = icon("download"))
    )
  ),
  dashboardBody(
    tags$head(
      tags$style(HTML("
        .shiny-input-container { margin-bottom: 15px; }
        .progress-bar { background-color: #367fa9; }
        .help-block { color: #666; font-size: 12px; }
      "))
    ),
    tabItems(
      tabItem(tabName = "upload",
              fluidRow(
                box(title = "Upload de Arquivos", width = 12, status = "primary",
                    helpText("Tamanho máximo por arquivo: 500MB"),

                    numericInput("num_images", "Quantas imagens Landsat deseja analisar?",
                                 value = 2, min = 1, max = 10, step = 1),

                    uiOutput("landsat_inputs"),

                    fileInput("geojson", "Selecione o arquivo GeoJSON do Município",
                              accept = c(".geojson", ".json"),
                              multiple = FALSE),

                    numericInput("temp_cutoff",
                                 "Temperatura de Corte para Áreas Quentes (°C):",
                                 value = 35,
                                 min = 0,
                                 max = 60,
                                 step = 0.5),

                    helpText("Defina a temperatura mínima para considerar como área quente."),
                    helpText("Valores acima deste limite serão classificados como áreas quentes."),

                    actionButton("process", "Processar Imagens",
                                 class = "btn-primary",
                                 icon = icon("cogs")),

                    helpText("Arquivos Landsat Collection 2 Level 2: Já incluem temperatura de superfície"),
                    helpText("Formato: Kelvin → Convertendo para Celsius"),
                    helpText("Sistemas de coordenadas devem ser compatíveis. Saiba mais em: https://www.usgs.gov/landsat-missions/landsat-surface-temperature")
                )
              )
      ),

      tabItem(tabName = "results",
              fluidRow(
                box(title = "Informações das Imagens", width = 12, status = "info",
                    verbatimTextOutput("image_info")
                )
              ),
              fluidRow(
                box(title = "Configuração e Progresso", width = 12, status = "info",
                    verbatimTextOutput("config_info"),
                    verbatimTextOutput("process_status")
                )
              ),
              fluidRow(
                box(title = "Temperaturas Médias", width = 6, status = "info",
                    tableOutput("temp_table")
                ),
                box(title = "Estatísticas de Temperatura", width = 6, status = "info",
                    verbatimTextOutput("temp_stats")
                )
              ),
              fluidRow(
                box(title = "Comparação entre Imagens", width = 12, status = "info",
                    plotOutput("comparison_plot")
                )
              ),
              # UI dinâmica para mapas
              uiOutput("dynamic_maps"),
              # UI dinâmica para gráficos de município
              uiOutput("dynamic_municipio_plots")
      ),

      tabItem(tabName = "download",
              fluidRow(
                box(title = "Download dos Resultados", width = 12, status = "success",
                    uiOutput("download_buttons"),
                    br(),
                    helpText("Temperatura em Celsius. Áreas quentes: acima do limite definido."),
                    helpText(paste("Limite atual:", textOutput("current_cutoff", inline = TRUE), "°C"))
                )
              )
      )
    )
  )
)

server <- function(input, output, session) {

  values <- reactiveValues(
    temp_rasters = list(),
    municipio_sf = NULL,
    hot_areas = list(),
    process_status = "Aguardando upload de arquivos...",
    image_names = c(),
    image_dates = c(),
    temp_cutoff = 35,
    processed = FALSE
  )

  # Gerar inputs dinâmicos para as imagens Landsat
  output$landsat_inputs <- renderUI({
    num_images <- input$num_images
    inputs <- tagList()

    for (i in 1:num_images) {
      inputs <- tagAppendChild(inputs,
                               fileInput(paste0("landsat", i),
                                         paste("Selecione a imagem Landsat", i, "(.tif)"),
                                         accept = c(".tif", ".tiff"),
                                         multiple = FALSE)
      )
    }

    inputs
  })

  # Observar mudanças no limite de temperatura
  observe({
    values$temp_cutoff <- input$temp_cutoff
  })

  # Output para mostrar o limite atual
  output$current_cutoff <- renderText({
    values$temp_cutoff
  })

  # Informações de configuração
  output$config_info <- renderPrint({
    cat("Configuração atual:\n")
    cat("Número de imagens:", input$num_images, "\n")
    cat("Temperatura de corte:", values$temp_cutoff, "°C\n")
    cat("Áreas quentes: temperaturas >", values$temp_cutoff, "°C\n")
  })

  # Extrair data do nome do arquivo Landsat
  extract_landsat_date <- function(filename) {
    date_pattern <- "\\d{8}"
    dates <- str_extract_all(filename, date_pattern)[[1]]

    if (length(dates) >= 1) {
      date_str <- dates[1]
      year <- substr(date_str, 1, 4)
      month <- substr(date_str, 5, 6)

      month_names <- c("Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
                       "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro")

      month_num <- as.numeric(month)
      if (month_num >= 1 && month_num <= 12) {
        return(paste(month_names[month_num], year))
      }
    }

    return("Data_Desconhecida")
  }

  # Função para verificar se todos os arquivos estão prontos
  check_files_ready <- reactive({
    num_images <- input$num_images
    all_ready <- !is.null(input$geojson)

    if (all_ready && num_images > 0) {
      for (i in 1:num_images) {
        file_input <- input[[paste0("landsat", i)]]
        if (is.null(file_input)) {
          all_ready <- FALSE
          break
        }
      }
    }

    return(all_ready)
  })

  # Observar upload de arquivos
  observe({
    if (check_files_ready()) {
      num_images <- input$num_images

      image_names <- character(num_images)
      image_dates <- character(num_images)

      for (i in 1:num_images) {
        file_input <- input[[paste0("landsat", i)]]
        image_names[i] <- file_input$name
        image_dates[i] <- extract_landsat_date(file_input$name)
      }

      values$image_names <- image_names
      values$image_dates <- image_dates
      values$process_status <- paste("Arquivos carregados. Clique em 'Processar Imagens'.")
    } else {
      values$process_status <- "Aguardando upload de arquivos..."
    }
  })

  # Função para cálculo de temperatura
  calculate_temperature_celsius <- function(raster_layer) {
    tryCatch({
      if (is.int(raster_layer)) {
        temp_k <- 0.00341802 * raster_layer + 149.0
      } else {
        temp_k <- raster_layer
      }

      temp_c <- temp_k - 273.15
      temp_c[temp_c < -50 | temp_c > 60] <- NA

      return(temp_c)

    }, error = function(e) {
      showNotification(paste("Erro no cálculo de temperatura:", e$message), type = "error")
      return(NULL)
    })
  }

  # Função para identificar áreas quentes
  identify_hot_areas <- function(temp_raster, cutoff_temp) {
    tryCatch({
      hot_mask <- temp_raster > cutoff_temp
      hot_polygons <- as.polygons(hot_mask)
      hot_areas <- hot_polygons[hot_polygons$lyr.1 == 1, ]

      if (nrow(hot_areas) > 0) {
        hot_areas_sf <- st_as_sf(hot_areas)
        hot_areas_sf <- st_set_crs(hot_areas_sf, st_crs(temp_raster))
        hot_areas_sf <- st_make_valid(hot_areas_sf)
        return(hot_areas_sf)
      } else {
        return(NULL)
      }

    }, error = function(e) {
      showNotification(paste("Erro ao identificar áreas quentes:", e$message), type = "error")
      return(NULL)
    })
  }

  # Função para calcular área em km²
  calculate_area_km2 <- function(sf_object) {
    tryCatch({
      if (is.null(sf_object) || nrow(sf_object) == 0) {
        return(0)
      }

      area_m2 <- as.numeric(sum(st_area(sf_object)))
      area_km2 <- area_m2 / 1e6
      return(round(area_km2, 2))

    }, error = function(e) {
      return(NA)
    })
  }

  observeEvent(input$process, {
    req(input$geojson, input$temp_cutoff)

    if (!check_files_ready()) {
      showNotification("Por favor, faça upload de todos os arquivos necessários.", type = "warning")
      return()
    }

    num_images <- input$num_images

    showModal(modalDialog(
      title = "Processando imagens...",
      paste("Processando", num_images, "imagens com temperatura de corte:", input$temp_cutoff, "°C"),
      footer = NULL,
      easyClose = FALSE
    ))

    tryCatch({
      values$process_status <- "Carregando GeoJSON..."

      # Carregar GeoJSON
      values$municipio_sf <- st_read(input$geojson$datapath, quiet = TRUE)
      values$municipio_sf <- st_make_valid(values$municipio_sf)

      if (is.null(values$municipio_sf)) {
        stop("Não foi possível carregar o arquivo GeoJSON")
      }

      # Inicializar listas
      temp_rasters <- list()
      hot_areas <- list()

      for (i in 1:num_images) {
        values$process_status <- paste("Carregando Landsat", i, "...")

        # Carregar imagem Landsat
        landsat_file <- input[[paste0("landsat", i)]]
        landsat_rast <- rast(landsat_file$datapath)

        # Converter GeoJSON para o mesmo CRS das imagens
        landsat_crs <- crs(landsat_rast)
        municipio_transformed <- st_transform(values$municipio_sf, landsat_crs)

        values$process_status <- paste("Recortando Landsat", i, "...")

        # Recortar pelo município
        landsat_cropped <- crop(landsat_rast, municipio_transformed)
        landsat_masked <- mask(landsat_cropped, municipio_transformed)

        values$process_status <- paste("Calculando temperatura Landsat", i, "...")

        # Encontrar banda de temperatura
        band_names <- names(landsat_masked)
        thermal_band <- which(grepl("ST_B10|ST", band_names, ignore.case = TRUE))[1]

        if (is.na(thermal_band)) thermal_band <- 1

        # Calcular temperatura em Celsius
        temp_celsius <- calculate_temperature_celsius(landsat_masked[[thermal_band]])

        if (!is.null(temp_celsius)) {
          names(temp_celsius) <- "Temperatura_Celsius"
          temp_rasters[[i]] <- temp_celsius

          values$process_status <- paste("Identificando áreas quentes Landsat", i, "...")

          # Identificar áreas mais quentes
          hot_areas[[i]] <- identify_hot_areas(temp_celsius, input$temp_cutoff)
        }
      }

      # Atualizar valores reativos
      values$temp_rasters <- temp_rasters
      values$hot_areas <- hot_areas
      values$processed <- TRUE

      values$process_status <- paste("Processamento concluído com sucesso! Limite:", input$temp_cutoff, "°C")
      removeModal()
      showNotification(paste("Processamento concluído! Limite:", input$temp_cutoff, "°C"), type = "message")

    }, error = function(e) {
      removeModal()
      values$process_status <- paste("Erro:", e$message)
      showNotification(paste("Erro:", e$message), type = "error")
    })
  })

  output$image_info <- renderPrint({
    req(values$image_names, values$image_dates)
    cat("Arquivos carregados:\n")
    for (i in 1:length(values$image_names)) {
      cat(paste0("Landsat ", i, ": ", values$image_names[i],
                 " (", values$image_dates[i], ")\n"))
    }
    cat("Tipo: Landsat Collection 2 Level 2\n")
    cat("Banda: ST_B10 (Temperatura de Superfície em Kelvin)\n")
    cat("Conversão: Kelvin → Celsius (K - 273.15)\n")
  })

  output$process_status <- renderText({
    values$process_status
  })

  # UI dinâmica para mapas
  output$dynamic_maps <- renderUI({
    req(values$processed, length(values$temp_rasters) > 0)

    num_images <- length(values$temp_rasters)
    map_boxes <- lapply(1:num_images, function(i) {
      box(
        title = paste("Mapa de Temperatura -", values$image_dates[i]),
        width = 6, status = "info",
        leafletOutput(paste0("map", i), height = 400)
      )
    })

    fluidRow(do.call(tagList, map_boxes))
  })

  # UI dinâmica para gráficos de município
  output$dynamic_municipio_plots <- renderUI({
    req(values$processed, values$municipio_sf)

    num_images <- length(values$temp_rasters)
    plot_boxes <- lapply(1:num_images, function(i) {
      box(
        title = paste("Município e Áreas Quentes -", values$image_dates[i]),
        width = 6, status = "info",
        plotOutput(paste0("municipio_plot", i), height = 400)
      )
    })

    fluidRow(do.call(tagList, plot_boxes))
  })

  # Gerar mapas dinamicamente
  observe({
    req(values$processed, values$temp_rasters, values$municipio_sf)

    num_images <- length(values$temp_rasters)

    for (i in 1:num_images) {
      local({
        local_i <- i
        output[[paste0("map", local_i)]] <- renderLeaflet({
          req(values$temp_rasters[[local_i]])

          tryCatch({
            temp_raster <- values$temp_rasters[[local_i]]

            # Converter para WGS84
            temp_raster_wgs84 <- project(temp_raster, "EPSG:4326", method = "bilinear")
            municipio_wgs84 <- st_transform(values$municipio_sf, 4326)

            # Extrair valores para a paleta de cores
            vals <- values(temp_raster_wgs84)
            vals <- vals[!is.na(vals)]

            if (length(vals) == 0) {
              return(leaflet() %>% addTiles() %>% addControl("Dados não disponíveis", position = "topright"))
            }

            pal <- colorNumeric(
              palette = c("blue", "cyan", "green", "yellow", "orange", "red"),
              domain = vals,
              na.color = "transparent"
            )

            # Criar mapa leaflet
            map <- leaflet() %>%
              addTiles() %>%
              addRasterImage(temp_raster_wgs84, colors = pal, opacity = 0.8) %>%
              addPolygons(data = municipio_wgs84, color = "black", weight = 2,
                          fillOpacity = 0, opacity = 1) %>%
              addLegend(pal = pal, values = vals,
                        title = paste("Temperatura (°C) -", values$image_dates[local_i]))

            # Adicionar áreas quentes se existirem
            if (length(values$hot_areas) >= local_i &&
                !is.null(values$hot_areas[[local_i]]) &&
                nrow(values$hot_areas[[local_i]]) > 0) {
              hot_areas_wgs84 <- st_transform(values$hot_areas[[local_i]], 4326)
              map <- map %>%
                addPolygons(data = hot_areas_wgs84, color = "darkred", weight = 2,
                            fillColor = "red", fillOpacity = 0.7,
                            popup = paste("Temperatura >", values$temp_cutoff, "°C"))
            }

            map

          }, error = function(e) {
            leaflet() %>% addTiles() %>% addControl("Erro ao carregar mapa", position = "topright")
          })
        })
      })
    }
  })

  # Gerar gráficos de município dinamicamente - CORRIGIDO
  observe({
    req(values$processed, values$municipio_sf)

    num_images <- length(values$temp_rasters)

    for (i in 1:num_images) {
      local({
        local_i <- i
        output[[paste0("municipio_plot", local_i)]] <- renderPlot({
          req(values$municipio_sf)

          # Converter para o mesmo CRS para evitar problemas de projeção
          municipio_plot <- st_transform(values$municipio_sf, 4326)

          p <- ggplot() +
            geom_sf(data = municipio_plot, fill = "lightblue", color = "darkblue", alpha = 0.5) +
            theme_minimal() +
            labs(title = paste("Município -", values$image_dates[local_i]),
                 subtitle = paste("Limite de áreas quentes:", values$temp_cutoff, "°C"))

          # Adicionar áreas quentes se existirem
          if (length(values$hot_areas) >= local_i &&
              !is.null(values$hot_areas[[local_i]]) &&
              nrow(values$hot_areas[[local_i]]) > 0) {

            # Converter áreas quentes para o mesmo CRS
            hot_areas_plot <- st_transform(values$hot_areas[[local_i]], 4326)
            hot_areas_plot <- st_make_valid(hot_areas_plot)

            p <- p + geom_sf(data = hot_areas_plot, fill = "red", color = "darkred", alpha = 0.7)
          }

          print(p)
        })
      })
    }
  })

  output$temp_table <- renderTable({
    req(values$processed, values$temp_rasters, values$municipio_sf)

    num_images <- length(values$temp_rasters)
    result_df <- data.frame()

    for (i in 1:num_images) {
      if (length(values$temp_rasters) >= i && !is.null(values$temp_rasters[[i]])) {
        temp_vals <- values(values$temp_rasters[[i]])
        temp_vals <- temp_vals[!is.na(temp_vals)]

        if (length(temp_vals) > 0) {
          # Calcular área total do município
          total_area_km2 <- calculate_area_km2(values$municipio_sf)

          # Calcular área quente
          hot_area_km2 <- if (length(values$hot_areas) >= i &&
                              !is.null(values$hot_areas[[i]]) &&
                              nrow(values$hot_areas[[i]]) > 0) {
            calculate_area_km2(values$hot_areas[[i]])
          } else 0

          # Calcular porcentagem
          area_quente_pct <- if (total_area_km2 > 0) {
            round((hot_area_km2 / total_area_km2) * 100, 2)
          } else 0

          result_df <- rbind(result_df, data.frame(
            Imagem = values$image_dates[i],
            Temperatura_Média = round(mean(temp_vals, na.rm = TRUE), 2),
            Temperatura_Máxima = round(max(temp_vals, na.rm = TRUE), 2),
            Temperatura_Mínima = round(min(temp_vals, na.rm = TRUE), 2),
            Área_Quente_km2 = hot_area_km2,
            Área_Quente_Percentual = area_quente_pct
          ))
        }
      }
    }

    result_df
  })

  output$temp_stats <- renderPrint({
    req(values$processed, values$temp_rasters)

    num_images <- length(values$temp_rasters)

    for (i in 1:num_images) {
      if (length(values$temp_rasters) >= i && !is.null(values$temp_rasters[[i]])) {
        temp_vals <- values(values$temp_rasters[[i]])
        temp_vals <- temp_vals[!is.na(temp_vals)]

        if (length(temp_vals) > 0) {
          cat(paste0("=== ", values$image_dates[i], " ===\n"))
          cat("Média:", round(mean(temp_vals, na.rm = TRUE), 2), "°C\n")
          cat("Mínima:", round(min(temp_vals, na.rm = TRUE), 2), "°C\n")
          cat("Máxima:", round(max(temp_vals, na.rm = TRUE), 2), "°C\n")
          cat("Desvio Padrão:", round(sd(temp_vals, na.rm = TRUE), 2), "°C\n")
          cat("Limite de áreas quentes:", values$temp_cutoff, "°C\n\n")
        }
      }
    }
  })

  output$comparison_plot <- renderPlot({
    req(values$processed, values$temp_rasters, length(values$temp_rasters) > 1)

    num_images <- length(values$temp_rasters)
    temp_data <- data.frame()

    for (i in 1:num_images) {
      if (length(values$temp_rasters) >= i && !is.null(values$temp_rasters[[i]])) {
        temp_vals <- values(values$temp_rasters[[i]])
        temp_vals <- temp_vals[!is.na(temp_vals)]

        if (length(temp_vals) > 0) {
          temp_data <- rbind(temp_data,
                             data.frame(Temperatura = temp_vals,
                                        Imagem = values$image_dates[i]))
        }
      }
    }

    if (nrow(temp_data) == 0) {
      return(ggplot() +
               annotate("text", x = 0.5, y = 0.5, label = "Dados insuficientes para plotar") +
               theme_void())
    }

    ggplot(temp_data, aes(x = Imagem, y = Temperatura, fill = Imagem)) +
      geom_boxplot(alpha = 0.7) +
      geom_hline(yintercept = values$temp_cutoff, linetype = "dashed", color = "red", size = 1) +
      annotate("text", x = 1, y = values$temp_cutoff + 2,
               label = paste("Limite:", values$temp_cutoff, "°C"), color = "red") +
      theme_minimal() +
      labs(title = "Comparação de Temperaturas entre Imagens",
           x = "Imagem", y = "Temperatura (°C)") +
      theme(legend.position = "none",
            axis.text.x = element_text(angle = 45, hjust = 1))
  })

  # Gerar botões de download dinamicamente
  output$download_buttons <- renderUI({
    req(values$processed, values$temp_rasters)

    num_images <- length(values$temp_rasters)
    buttons <- tagList()

    for (i in 1:num_images) {
      buttons <- tagAppendChild(buttons,
                                downloadButton(paste0("download_shp", i),
                                               paste("Download GeoJSON Áreas Quentes -", values$image_dates[i]))
      )
      buttons <- tagAppendChild(buttons,
                                downloadButton(paste0("download_map", i),
                                               paste("Download Mapa", values$image_dates[i], "(PNG)"))
      )
      buttons <- tagAppendChild(buttons,
                                downloadButton(paste0("download_raster", i),
                                               paste("Download Raster Temperatura -", values$image_dates[i], "(GeoTIFF)"))
      )
    }

    buttons
  })

  # Gerar handlers de download dinamicamente
  observe({
    req(values$processed, values$hot_areas)

    num_images <- length(values$hot_areas)

    for (i in 1:num_images) {
      local({
        local_i <- i
        output[[paste0("download_shp", local_i)]] <- downloadHandler(
          filename = function() {
            paste0("areas_quentes_", gsub(" ", "_", values$image_dates[local_i]), "_",
                   values$temp_cutoff, "C.geojson")
          },
          content = function(file) {
            if (length(values$hot_areas) >= local_i && !is.null(values$hot_areas[[local_i]])) {
              st_write(values$hot_areas[[local_i]], file, driver = "GeoJSON", delete_dsn = TRUE)
            }
          }
        )
      })
    }
  })

  observe({
    req(values$processed, values$temp_rasters)

    num_images <- length(values$temp_rasters)

    for (i in 1:num_images) {
      local({
        local_i <- i
        output[[paste0("download_map", local_i)]] <- downloadHandler(
          filename = function() {
            paste0("mapa_temperatura_", gsub(" ", "_", values$image_dates[local_i]), ".png")
          },
          content = function(file) {
            if (length(values$temp_rasters) >= local_i && !is.null(values$temp_rasters[[local_i]])) {
              png(file, width = 1000, height = 800)
              plot(values$temp_rasters[[local_i]],
                   main = paste("Temperatura", values$image_dates[local_i], "(°C)\nLimite:", values$temp_cutoff, "°C"))
              dev.off()
            }
          }
        )
      })
    }
  })

  # Handler para download do raster em Celsius (NOVO)
  observe({
    req(values$processed, values$temp_rasters)

    num_images <- length(values$temp_rasters)

    for (i in 1:num_images) {
      local({
        local_i <- i
        output[[paste0("download_raster", local_i)]] <- downloadHandler(
          filename = function() {
            paste0("temperatura_celsius_", gsub(" ", "_", values$image_dates[local_i]),
                   "_limite_", values$temp_cutoff, "C.tif")
          },
          content = function(file) {
            if (length(values$temp_rasters) >= local_i && !is.null(values$temp_rasters[[local_i]])) {
              tryCatch({
                # Escrever o raster em formato GeoTIFF
                writeRaster(values$temp_rasters[[local_i]], file,
                            filetype = "GTiff", overwrite = TRUE,
                            NAflag = -9999,
                            gdal = c("COMPRESS=LZW", "PREDICTOR=2"))

                showNotification(paste("Raster salvo com sucesso:", file), type = "message")

              }, error = function(e) {
                showNotification(paste("Erro ao salvar raster:", e$message), type = "error")
              })
            }
          }
        )
      })
    }
  })
}

shinyApp(ui, server)